In [1]:
%pip install pandas sentence-transformers llama-index transformers



  Using cached scipy-1.14.1-cp311-cp311-win_amd64.whl.metadata (60 kB)
     ---------------------------------------- 0.0/61.0 kB ? eta -:--:--
     ---------------------------------------- 61.0/61.0 kB 1.6 MB/s eta 0:00:00
  Using cached attrs-24.2.0-py3-none-any.whl.metadata (11 kB)
     ---------------------------------------- 0.0/66.2 kB ? eta -:--:--
     ----------------------- -------------- 41.0/66.2 kB 991.0 kB/s eta 0:00:01
     ---------------------------------------- 66.2/66.2 kB 1.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/11.6 MB ? eta -:--:--
   ---------------------------------------- 0.1/11.6 MB 3.3 MB/s eta 0:00:04
    --------------------------------------- 0.3/11.6 MB 3.4 MB/s eta 0:00:04
   - -------------------------------------- 0.4/11.6 MB 3.4 MB/s eta 0:00:04
   -- ------------------------------------- 0.6/11.6 MB 3.6 MB/s eta 0:00:04
   -- ------------------------------------- 0.8/11.6 MB 3.8 MB/s eta 0:00:03
   --- ---------------------


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from transformers import LlamaForCausalLM, LlamaTokenizer

model = LlamaForCausalLM.from_pretrained("meta-llama/Meta-Llama-3-8B")
tokenizer = LlamaTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B")

d:\project\python\thrainer\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


: 

In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np

# Inisialisasi model Sentence Transformer buat embedding
embedder = SentenceTransformer('all-MiniLM-L6-v2')

# Assuming df is defined in a previous cell
# Buat embedding untuk setiap deskripsi di CSV
df['embedding'] = df['Description'].apply(lambda x: embedder.encode(x))

# Simpan embedding untuk keperluan selanjutnya
df.head()



In [ ]:
def find_relevant_description(question, df, embedder):
    # Buat embedding dari pertanyaan
    question_embedding = embedder.encode(question)
    
    # Hitung kesamaan (cosine similarity) antara pertanyaan dan deskripsi
    similarities = df['embedding'].apply(
        lambda x: np.dot(x, question_embedding) / 
                  (np.linalg.norm(x) * np.linalg.norm(question_embedding))
    )
    
    # Ambil baris dengan similarity tertinggi
    best_match = df.iloc[similarities.argmax()]
    return best_match['Category'], best_match['Description']



In [ ]:
from transformers import LlamaForCausalLM, LlamaTokenizer

# Inisialisasi model Llama dan tokenizer
model = LlamaForCausalLM.from_pretrained("meta-llama/Llama-2-7b")
tokenizer = LlamaTokenizer.from_pretrained("meta-llama/Llama-2-7b")

def ask_llama(question, category, description, model, tokenizer):
    # Format prompt untuk model
    prompt = f"User asked: '{question}'.\n" \
             f"Here is the relevant information:\n" \
             f"Category: {category}\nDescription: {description}\n" \
             "Based on this information, provide a helpful answer."

    # Tokenisasi dan generate jawaban
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(**inputs, max_length=100)
    
    # Decode hasilnya
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


In [ ]:
# Pertanyaan dari user
question = "What services does your company provide?"

# Cari deskripsi yang relevan
category, description = find_relevant_description(question, df, embedder)

# Dapatkan jawaban dari Llama
response = ask_llama(question, category, description, model, tokenizer)

# Tampilkan jawaban
print(response)
